In [1]:
import pandas as pd

class Contract:
    
    def __init__(self, wholesale_price, revenue_share, buy_back_price):
        self.wholesale_price = wholesale_price
        self.revenue_share = revenue_share
        self.buy_back_price = buy_back_price
        
        self.retail_price = 1
        self.printing_cost = .45
        
        self.demand_frame = pd.DataFrame({'Demand':range(100 ,200)})
        self.demand_frame['Probability'] = 1/100
        
    def vendor_marginal_profit(self):
        return self.retail_price - self.wholesale_price - (1 - self.revenue_share)*self.retail_price
    
    def vendor_marginal_loss(self):
        return self.wholesale_price - self.buy_back_price
    
    def optimal_service(self):
        mp = self.vendor_marginal_profit()
        ml = self.vendor_marginal_loss()
        return mp/(mp+ml)
    
    def optimal_quantity(self):
        return round(100 + self.optimal_service()*100, 0)
    
    def vendor_daily_profit(self):
        return self.optimal_quantity()*self.vendor_marginal_profit()
    
    def daily_view(self):
        quantity = self.optimal_quantity()
        self.demand_frame['Number Sold'] = self.demand_frame['Demand'].clip(100, quantity)
        self.demand_frame['Surplus'] = (quantity - self.demand_frame['Demand']).clip(0, None)
        self.demand_frame['Vendor Revenue'] = self.demand_frame['Number Sold']*self.retail_price
        self.demand_frame['Buy-Back Amount'] = self.demand_frame['Surplus']*self.buy_back_price
        self.demand_frame['Revenue Sharing'] = self.demand_frame['Vendor Revenue']*(1 - self.revenue_share)
        
        self.demand_frame['Vendor Profit'] =  self.demand_frame['Vendor Revenue'] \
                                             -quantity*self.wholesale_price\
                                             -self.demand_frame['Revenue Sharing']\
                                             +self.demand_frame['Buy-Back Amount']       
        return self.demand_frame
    
    def vendor_expected_profit(self):
        df = self.daily_view()
        return (df['Vendor Profit']*df['Probability']).sum()
    
    def publisher_expected_profit(self):
        df = self.daily_view()
        quantity = self.optimal_quantity()
        
        return quantity*(self.wholesale_price - self.printing_cost)\
              -(df['Buy-Back Amount']*df['Probability']).sum()\
              +(df['Revenue Sharing']*df['Probability']).sum()  
    
    def supply_chain_expected_profit(self):
        return self.vendor_expected_profit()+self.publisher_expected_profit()

Standard:

In [2]:
contract = Contract(wholesale_price = .8,
                    revenue_share = 1,
                    buy_back_price = 0)

contract.daily_view()

,Demand,Probability,Number Sold,Surplus,Vendor Revenue,Buy-Back Amount,Revenue Sharing,Vendor Profit
0,100,0.01,100,20.0,100,0.0,0,4.0
1,101,0.01,101,19.0,101,0.0,0,5.0
2,102,0.01,102,18.0,102,0.0,0,6.0
3,103,0.01,103,17.0,103,0.0,0,7.0
4,104,0.01,104,16.0,104,0.0,0,8.0
5,105,0.01,105,15.0,105,0.0,0,9.0
6,106,0.01,106,14.0,106,0.0,0,10.0
7,107,0.01,107,13.0,107,0.0,0,11.0
8,108,0.01,108,12.0,108,0.0,0,12.0
9,109,0.01,109,11.0,109,0.0,0,13.0


In [3]:
print('Vendor MP: $', round(contract.vendor_marginal_profit(), 2), sep = '')
print('Vendor ML: $', round(contract.vendor_marginal_loss(), 2), sep = '')
print('Optimal Service: ', round(100*contract.optimal_service(), 2), '%', sep = '')
print('Optimal Order Quantity:', contract.optimal_quantity())
print('Vendor Expected Daily Profit: $', round(contract.vendor_expected_profit(), 2), sep = '')
print('Publisher Expected Daily Profit: $', round(contract.publisher_expected_profit(), 2), sep = '')
print('Expected Supply Chain Daily Profit: $', round(contract.supply_chain_expected_profit(), 2), sep = '')

Vendor MP: $0.2
Vendor ML: $0.8
Optimal Service: 20.0%
Optimal Order Quantity: 120.0
Vendor Expected Daily Profit: $21.9
Publisher Expected Daily Profit: $42.0
Expected Supply Chain Daily Profit: $63.9


Revenue Sharing:

In [4]:
contract = Contract(wholesale_price = .45,
                    revenue_share = .65,
                    buy_back_price = 0)

contract.daily_view()

,Demand,Probability,Number Sold,Surplus,Vendor Revenue,Buy-Back Amount,Revenue Sharing,Vendor Profit
0,100,0.01,100,31.0,100,0.0,35.00,6.05
1,101,0.01,101,30.0,101,0.0,35.35,6.70
2,102,0.01,102,29.0,102,0.0,35.70,7.35
3,103,0.01,103,28.0,103,0.0,36.05,8.00
4,104,0.01,104,27.0,104,0.0,36.40,8.65
5,105,0.01,105,26.0,105,0.0,36.75,9.30
6,106,0.01,106,25.0,106,0.0,37.10,9.95
7,107,0.01,107,24.0,107,0.0,37.45,10.60
8,108,0.01,108,23.0,108,0.0,37.80,11.25
9,109,0.01,109,22.0,109,0.0,38.15,11.90


In [5]:
print('Vendor MP: $', round(contract.vendor_marginal_profit(), 2), sep = '')
print('Vendor ML: $', round(contract.vendor_marginal_loss(), 2), sep = '')
print('Optimal Service: ', round(100*contract.optimal_service(), 2), '%', sep = '')
print('Optimal Order Quantity:', contract.optimal_quantity())
print('Vendor Expected Daily Profit: $', round(contract.vendor_expected_profit(), 2), sep = '')
print('Publisher Expected Daily Profit: $', round(contract.publisher_expected_profit(), 2), sep = '')
print('Expected Supply Chain Daily Profit: $', round(contract.supply_chain_expected_profit(), 2), sep = '')

Vendor MP: $0.2
Vendor ML: $0.45
Optimal Service: 30.77%
Optimal Order Quantity: 131.0
Vendor Expected Daily Profit: $22.98
Publisher Expected Daily Profit: $44.11
Expected Supply Chain Daily Profit: $67.09


Buy-Back (1)

In [6]:
contract = Contract(wholesale_price = .8,
                    revenue_share = 1,
                    buy_back_price = .6)

contract.daily_view()

,Demand,Probability,Number Sold,Surplus,Vendor Revenue,Buy-Back Amount,Revenue Sharing,Vendor Profit
0,100,0.01,100,50.0,100,30.0,0,10.0
1,101,0.01,101,49.0,101,29.4,0,10.4
2,102,0.01,102,48.0,102,28.8,0,10.8
3,103,0.01,103,47.0,103,28.2,0,11.2
4,104,0.01,104,46.0,104,27.6,0,11.6
5,105,0.01,105,45.0,105,27.0,0,12.0
6,106,0.01,106,44.0,106,26.4,0,12.4
7,107,0.01,107,43.0,107,25.8,0,12.8
8,108,0.01,108,42.0,108,25.2,0,13.2
9,109,0.01,109,41.0,109,24.6,0,13.6


In [7]:
print('Vendor MP: $', round(contract.vendor_marginal_profit(), 2), sep = '')
print('Vendor ML: $', round(contract.vendor_marginal_loss(), 2), sep = '')
print('Optimal Service: ', round(100*contract.optimal_service(), 2), '%', sep = '')
print('Optimal Order Quantity:', contract.optimal_quantity())
print('Vendor Expected Daily Profit: $', round(contract.vendor_expected_profit(), 2), sep = '')
print('Publisher Expected Daily Profit: $', round(contract.publisher_expected_profit(), 2), sep = '')
print('Expected Supply Chain Daily Profit: $', round(contract.supply_chain_expected_profit(), 2), sep = '')

Vendor MP: $0.2
Vendor ML: $0.2
Optimal Service: 50.0%
Optimal Order Quantity: 150.0
Vendor Expected Daily Profit: $24.9
Publisher Expected Daily Profit: $44.85
Expected Supply Chain Daily Profit: $69.75


Buy-Back (2)

In [8]:
contract = Contract(wholesale_price = .8,
                    revenue_share = 1,
                    buy_back_price = .6363)

contract.daily_view()

,Demand,Probability,Number Sold,Surplus,Vendor Revenue,Buy-Back Amount,Revenue Sharing,Vendor Profit
0,100,0.01,100,55.0,100,34.9965,0,10.9965
1,101,0.01,101,54.0,101,34.3602,0,11.3602
2,102,0.01,102,53.0,102,33.7239,0,11.7239
3,103,0.01,103,52.0,103,33.0876,0,12.0876
4,104,0.01,104,51.0,104,32.4513,0,12.4513
5,105,0.01,105,50.0,105,31.8150,0,12.8150
6,106,0.01,106,49.0,106,31.1787,0,13.1787
7,107,0.01,107,48.0,107,30.5424,0,13.5424
8,108,0.01,108,47.0,108,29.9061,0,13.9061
9,109,0.01,109,46.0,109,29.2698,0,14.2698


In [9]:
print('Vendor MP: $', round(contract.vendor_marginal_profit(), 2), sep = '')
print('Vendor ML: $', round(contract.vendor_marginal_loss(), 2), sep = '')
print('Optimal Service: ', round(100*contract.optimal_service(), 2), '%', sep = '')
print('Optimal Order Quantity:', contract.optimal_quantity())
print('Vendor Expected Daily Profit: $', round(contract.vendor_expected_profit(), 2), sep = '')
print('Publisher Expected Daily Profit: $', round(contract.publisher_expected_profit(), 2), sep = '')
print('Expected Supply Chain Daily Profit: $', round(contract.supply_chain_expected_profit(), 2), sep = '')

Vendor MP: $0.2
Vendor ML: $0.16
Optimal Service: 54.99%
Optimal Order Quantity: 155.0
Vendor Expected Daily Profit: $25.4
Publisher Expected Daily Profit: $44.45
Expected Supply Chain Daily Profit: $69.85


Buy-Back (3)

In [10]:
contract = Contract(wholesale_price = .6,
                    revenue_share = 1,
                    buy_back_price = .2727)

contract.daily_view()

,Demand,Probability,Number Sold,Surplus,Vendor Revenue,Buy-Back Amount,Revenue Sharing,Vendor Profit
0,100,0.01,100,55.0,100,14.9985,0,21.9985
1,101,0.01,101,54.0,101,14.7258,0,22.7258
2,102,0.01,102,53.0,102,14.4531,0,23.4531
3,103,0.01,103,52.0,103,14.1804,0,24.1804
4,104,0.01,104,51.0,104,13.9077,0,24.9077
5,105,0.01,105,50.0,105,13.6350,0,25.6350
6,106,0.01,106,49.0,106,13.3623,0,26.3623
7,107,0.01,107,48.0,107,13.0896,0,27.0896
8,108,0.01,108,47.0,108,12.8169,0,27.8169
9,109,0.01,109,46.0,109,12.5442,0,28.5442


In [11]:
print('Vendor MP: $', round(contract.vendor_marginal_profit(), 2), sep = '')
print('Vendor ML: $', round(contract.vendor_marginal_loss(), 2), sep = '')
print('Optimal Service: ', round(100*contract.optimal_service(), 2), '%', sep = '')
print('Optimal Order Quantity:', contract.optimal_quantity())
print('Vendor Expected Daily Profit: $', round(contract.vendor_expected_profit(), 2), sep = '')
print('Publisher Expected Daily Profit: $', round(contract.publisher_expected_profit(), 2), sep = '')
print('Expected Supply Chain Daily Profit: $', round(contract.supply_chain_expected_profit(), 2), sep = '')

Vendor MP: $0.4
Vendor ML: $0.33
Optimal Service: 55.0%
Optimal Order Quantity: 155.0
Vendor Expected Daily Profit: $50.8
Publisher Expected Daily Profit: $19.05
Expected Supply Chain Daily Profit: $69.85
